In [0]:

# ## Ravish Chawla
# ### Topic Modeling with LDA and NMF algorithms on the ABC News Headlines Dataset
# #### July 31, 2017

# # Data imports
# 
# We import Pandas, numpy and scipy for data structures. We use gensim for LDA, and sklearn for NMF

# In[28]:

import pandas as pd;
import numpy as np;
import scipy as sp;
import sklearn;
import sys;
from nltk.corpus import stopwords;
import nltk;
from gensim.models import ldamodel
import gensim.corpora;
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer;
from sklearn.decomposition import NMF;
from sklearn.preprocessing import normalize;
import pickle;


# # Loading the data
# 
# We are using the ABC News headlines dataset. Some lines are badly formatted (very few), so we are skipping those.

# In[ ]:

data = pd.read_csv('en_interim_01_05_DV_IN_with_full_texts.csv', error_bad_lines=False);



In [9]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
data.head()

,Unnamed: 0,Unnamed: 0.1,index,GLOBALEVENTID,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,Actor1Religion1Code,Actor1Religion2Code,Actor1Type1Code,Actor1Type2Code,Actor1Type3Code,Actor2Code,Actor2Name,Actor2CountryCode,Actor2KnownGroupCode,Actor2EthnicCode,Actor2Religion1Code,Actor2Religion2Code,Actor2Type1Code,Actor2Type2Code,Actor2Type3Code,IsRootEvent,EventCode,CAMEOCodeDescription,EventBaseCode,EventRootCode,QuadClass,GoldsteinScale,NumMentions,NumSources,NumArticles,AvgTone,Actor1Geo_Type,Actor1Geo_FullName,Actor1Geo_CountryCode,Actor1Geo_ADM1Code,Actor1Geo_ADM2Code,Actor1Geo_Lat,Actor1Geo_Long,Actor1Geo_FeatureID,Actor2Geo_Type,Actor2Geo_FullName,Actor2Geo_CountryCode,Actor2Geo_ADM1Code,Actor2Geo_ADM2Code,Actor2Geo_Lat,Actor2Geo_Long,Actor2Geo_FeatureID,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_ADM2Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL,title,text
0,0,0,72,896050132,19200101,192001,1920,1920.0027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COP,POLICE,NaN,NaN,NaN,NaN,NaN,COP,NaN,NaN,1,81,"Ease administrative sanctions, not specified ...",81,8,2,5.0,4,1,4,-4.953560,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,"Surat, Gujarat, India",IN,IN09,70127,20.9667,72.9000,-2098110,4,"Surat, Gujarat, India",IN,IN09,70127.0,20.9667,72.9000,-2098110,20200101001500,https://timesofindia.indiatimes.com/city/surat...,who killed dishit wife two others set to walk ...,"Surat: Who killed Dishit Jariwala ? Wife, two ..."
1,1,1,11452,896059553,19200101,192001,1920,1920.0027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,USA,UNITED STATES,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,42,Make a visit,42,4,1,1.9,1,1,1,-2.774887,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,"Chicago, Illinois, United States",US,USIL,IL031,41.8500,-87.6501,423587,4,"Calcutta, West Bengal, India",IN,IN28,70294.0,22.5697,88.3697,-2092511,20200101010000,https://www.sheltonherald.com/news/crime/artic...,Lawsuit Famed Jesuit abused boy 1 000 times 14...,NaN
2,2,2,23149,896077202,19200101,192001,1920,1920.0027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COP,POLICE,NaN,NaN,NaN,NaN,NaN,COP,NaN,NaN,1,36,Express intent to meet or negotiate,36,3,1,4.0,4,1,4,-5.000000,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,"Muzaffarnagar, Uttar Pradesh, India",IN,IN36,17929,29.4667,77.6833,-2105244,4,"Muzaffarnagar, Uttar Pradesh, India",IN,IN36,17929.0,29.4667,77.6833,-2105244,20200101041500,http://www.canindia.com/caa-protests-shia-cler...,caa protests shia cleric kalbe jawad to meet f...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nCAA protests: Shia...
3,3,3,30812,896087593,19200101,192001,1920,1920.0027,GOV,GOVERNOR,NaN,NaN,NaN,NaN,NaN,GOV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,141,Demonstrate or rally,141,14,3,-6.5,2,1,2,-2.087475,4,"Kannur, Kerala, India",IN,IN13,17698,11.8667,75.3653,-2092528,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,"Kerala, Kerala, India",IN,IN13,17699.0,10.0000,76.5000,-2100307,20200101063000,https://www.business-standard.com/article/pti-...,entry of 2 young women in sabarimala rahul gan...,\n\n\n\n\n\n\n\nEntry of 2 young women in Sab...
4,4,4,31666,896088447,19200101,192001,1920,1920.0027,GOV,MINIST,NaN,NaN,NaN,NaN,NaN,GOV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,112,"Accuse, not specified below",112,11,3,-2.0,6,1,6,-3.986711,1,India,IN,IN,NaN,20.0000,77.0000,IN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,India,IN,IN,NaN,20.0000,77.0000,IN,20200101064500,https://thehimalayantimes.com/nepal/minister-g...,minister gokul prasad baskota pledges to book ...,\n\n\n\n\n\n\n\n\nMinister Gokul Prasad Baskot...


In [0]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(max_df=0.8, min_df=2, stop_words='english')
doc_term_matrix = count_vect.fit_transform(data['text'].values.astype('U'))

In [54]:
from sklearn.decomposition import LatentDirichletAllocation

LDA = LatentDirichletAllocation(n_components=5, random_state=42)
LDA.fit(doc_term_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [56]:
import random

for i in range(10):
    random_id = random.randint(0,len(count_vect.get_feature_names()))
    print(count_vect.get_feature_names()[random_id])

asymptomatic
newsbudget
specialises
sickness
firdous
kutcha
stating
8750871221
ebooks
rebalancing


In [61]:
for i,topic in enumerate(LDA.components_):
    print(f'Top 10 words for topic #{i}:')
    print([count_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

Top 10 words for topic #0:
['share', 'lockdown', 'orissapost', 'plastic', 'subscribe', '19', 'coronavirus', 'covid', 'odisha', 'free']


Top 10 words for topic #1:
['death', 'case', 'said', 'rape', 'convicts', 'covid', '19', 'ht', 'court', 'delhi']


Top 10 words for topic #2:
['policy', 'covid', 'law', '19', 'said', 'world', 'bs', 'business', 'new', 'women']


Top 10 words for topic #3:
['express', 'indian', 'cases', '19', 'tv', 'delhi', 'women', 'said', 'covid', 'police']


Top 10 words for topic #4:
['indiacoronavirus', 'policy', 'mubarak', 'cases', 'economic', 'police', 'said', 'helpline', 'times', 'coronavirus']




In [62]:
topic_values = LDA.transform(doc_term_matrix)
topic_values.shape

(6708, 5)

In [0]:
data['Topic_lda'] = topic_values.argmax(axis=1)

In [77]:
data.head()

,Unnamed: 0,Unnamed: 0.1,index,GLOBALEVENTID,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,Actor1Religion1Code,Actor1Religion2Code,Actor1Type1Code,Actor1Type2Code,Actor1Type3Code,Actor2Code,Actor2Name,Actor2CountryCode,Actor2KnownGroupCode,Actor2EthnicCode,Actor2Religion1Code,Actor2Religion2Code,Actor2Type1Code,Actor2Type2Code,Actor2Type3Code,IsRootEvent,EventCode,CAMEOCodeDescription,EventBaseCode,EventRootCode,QuadClass,GoldsteinScale,NumMentions,NumSources,NumArticles,AvgTone,Actor1Geo_Type,Actor1Geo_FullName,Actor1Geo_CountryCode,Actor1Geo_ADM1Code,Actor1Geo_ADM2Code,Actor1Geo_Lat,Actor1Geo_Long,Actor1Geo_FeatureID,Actor2Geo_Type,Actor2Geo_FullName,Actor2Geo_CountryCode,Actor2Geo_ADM1Code,Actor2Geo_ADM2Code,Actor2Geo_Lat,Actor2Geo_Long,Actor2Geo_FeatureID,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_ADM2Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL,title,text,Topic_nmf,Topic_lda
0,0,0,72,896050132,19200101,192001,1920,1920.0027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COP,POLICE,NaN,NaN,NaN,NaN,NaN,COP,NaN,NaN,1,81,"Ease administrative sanctions, not specified ...",81,8,2,5.0,4,1,4,-4.953560,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,"Surat, Gujarat, India",IN,IN09,70127,20.9667,72.9000,-2098110,4,"Surat, Gujarat, India",IN,IN09,70127.0,20.9667,72.9000,-2098110,20200101001500,https://timesofindia.indiatimes.com/city/surat...,who killed dishit wife two others set to walk ...,"Surat: Who killed Dishit Jariwala ? Wife, two ...",2,4
1,1,1,11452,896059553,19200101,192001,1920,1920.0027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,USA,UNITED STATES,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,42,Make a visit,42,4,1,1.9,1,1,1,-2.774887,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,"Chicago, Illinois, United States",US,USIL,IL031,41.8500,-87.6501,423587,4,"Calcutta, West Bengal, India",IN,IN28,70294.0,22.5697,88.3697,-2092511,20200101010000,https://www.sheltonherald.com/news/crime/artic...,Lawsuit Famed Jesuit abused boy 1 000 times 14...,NaN,2,4
2,2,2,23149,896077202,19200101,192001,1920,1920.0027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COP,POLICE,NaN,NaN,NaN,NaN,NaN,COP,NaN,NaN,1,36,Express intent to meet or negotiate,36,3,1,4.0,4,1,4,-5.000000,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,"Muzaffarnagar, Uttar Pradesh, India",IN,IN36,17929,29.4667,77.6833,-2105244,4,"Muzaffarnagar, Uttar Pradesh, India",IN,IN36,17929.0,29.4667,77.6833,-2105244,20200101041500,http://www.canindia.com/caa-protests-shia-cler...,caa protests shia cleric kalbe jawad to meet f...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nCAA protests: Shia...,0,0
3,3,3,30812,896087593,19200101,192001,1920,1920.0027,GOV,GOVERNOR,NaN,NaN,NaN,NaN,NaN,GOV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,141,Demonstrate or rally,141,14,3,-6.5,2,1,2,-2.087475,4,"Kannur, Kerala, India",IN,IN13,17698,11.8667,75.3653,-2092528,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,"Kerala, Kerala, India",IN,IN13,17699.0,10.0000,76.5000,-2100307,20200101063000,https://www.business-standard.com/article/pti-...,entry of 2 young women in sabarimala rahul gan...,\n\n\n\n\n\n\n\nEntry of 2 young women in Sab...,1,2
4,4,4,31666,896088447,19200101,192001,1920,1920.0027,GOV,MINIST,NaN,NaN,NaN,NaN,NaN,GOV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,112,"Accuse, not specified below",112,11,3,-2.0,6,1,6,-3.986711,1,India,IN,IN,NaN,20.0000,77.0000,IN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,India,IN,IN,NaN,20.0000,77.0000,IN,20200101064500,https://thehimalayantimes.com/nepal/minister-g...,minister gokul prasad baskota pledges to book ...,\n\n\n\n\n\n\n\n\nMinister Gokul Prasad Baskot...,0,3


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(max_df=0.8, min_df=2, stop_words='english')
doc_term_matrix = tfidf_vect.fit_transform(data['text'].values.astype('U'))

In [44]:
from sklearn.decomposition import NMF

nmf = NMF(n_components=5, random_state=42)
nmf.fit(doc_term_matrix )

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=5, random_state=42, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

In [46]:
import random

for i in range(10):
    random_id = random.randint(0,len(tfidf_vect.get_feature_names()))
    print(tfidf_vect.get_feature_names()[random_id])

licking
redundant
musings
noteworthy
hercules
screams
gabriella
liberated
affiliated
stimulate


In [0]:
first_topic = nmf.components_[0]
top_topic_words = first_topic.argsort()[-10:]

In [48]:
for i in top_topic_words:
    print(tfidf_vect.get_feature_names()[i])

said
hindu
delhi
day
police
lockdown
coronavirus
covid
19
women


In [49]:
for i,topic in enumerate(nmf.components_):
    print(f'Top 10 words for topic #{i}:')
    print([tfidf_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

Top 10 words for topic #0:
['said', 'hindu', 'delhi', 'day', 'police', 'lockdown', 'coronavirus', 'covid', '19', 'women']


Top 10 words for topic #1:
['strategistweekenddata', 'paperlatest', 'seriesipl', 'commentcolumnistsbs', 'toolsderivativesbs', 'worldsmestart', 'premium', 'fund', 'standard', 'bs']


Top 10 words for topic #2:
['indiaindia', '2020coronavirus', 'livepakistan', 'crashirctc', 'newstimes', 'economic', 'indiacoronavirus', 'times', 'mubarak', 'helpline']


Top 10 words for topic #3:
['north', 'chandigarh', 'tv', 'launch', 'indian', 'mask', 'viral', 'watch', 'realme', 'express']


Top 10 words for topic #4:
['https', 'gang', 'www', 'rape', 'html', 'hindustantimes', 'court', 'delhi', 'convicts', 'ht']




In [55]:
topic_values = nmf.transform(doc_term_matrix)
data['Topic_nmf'] = topic_values.argmax(axis=1)
data.head()

,Unnamed: 0,Unnamed: 0.1,index,GLOBALEVENTID,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,Actor1Religion1Code,Actor1Religion2Code,Actor1Type1Code,Actor1Type2Code,Actor1Type3Code,Actor2Code,Actor2Name,Actor2CountryCode,Actor2KnownGroupCode,Actor2EthnicCode,Actor2Religion1Code,Actor2Religion2Code,Actor2Type1Code,Actor2Type2Code,Actor2Type3Code,IsRootEvent,EventCode,CAMEOCodeDescription,EventBaseCode,EventRootCode,QuadClass,GoldsteinScale,NumMentions,NumSources,NumArticles,AvgTone,Actor1Geo_Type,Actor1Geo_FullName,Actor1Geo_CountryCode,Actor1Geo_ADM1Code,Actor1Geo_ADM2Code,Actor1Geo_Lat,Actor1Geo_Long,Actor1Geo_FeatureID,Actor2Geo_Type,Actor2Geo_FullName,Actor2Geo_CountryCode,Actor2Geo_ADM1Code,Actor2Geo_ADM2Code,Actor2Geo_Lat,Actor2Geo_Long,Actor2Geo_FeatureID,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_ADM2Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL,title,text,Topic,Topic_nmf
0,0,0,72,896050132,19200101,192001,1920,1920.0027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COP,POLICE,NaN,NaN,NaN,NaN,NaN,COP,NaN,NaN,1,81,"Ease administrative sanctions, not specified ...",81,8,2,5.0,4,1,4,-4.953560,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,"Surat, Gujarat, India",IN,IN09,70127,20.9667,72.9000,-2098110,4,"Surat, Gujarat, India",IN,IN09,70127.0,20.9667,72.9000,-2098110,20200101001500,https://timesofindia.indiatimes.com/city/surat...,who killed dishit wife two others set to walk ...,"Surat: Who killed Dishit Jariwala ? Wife, two ...",2,2
1,1,1,11452,896059553,19200101,192001,1920,1920.0027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,USA,UNITED STATES,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,42,Make a visit,42,4,1,1.9,1,1,1,-2.774887,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,"Chicago, Illinois, United States",US,USIL,IL031,41.8500,-87.6501,423587,4,"Calcutta, West Bengal, India",IN,IN28,70294.0,22.5697,88.3697,-2092511,20200101010000,https://www.sheltonherald.com/news/crime/artic...,Lawsuit Famed Jesuit abused boy 1 000 times 14...,NaN,2,2
2,2,2,23149,896077202,19200101,192001,1920,1920.0027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COP,POLICE,NaN,NaN,NaN,NaN,NaN,COP,NaN,NaN,1,36,Express intent to meet or negotiate,36,3,1,4.0,4,1,4,-5.000000,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,"Muzaffarnagar, Uttar Pradesh, India",IN,IN36,17929,29.4667,77.6833,-2105244,4,"Muzaffarnagar, Uttar Pradesh, India",IN,IN36,17929.0,29.4667,77.6833,-2105244,20200101041500,http://www.canindia.com/caa-protests-shia-cler...,caa protests shia cleric kalbe jawad to meet f...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nCAA protests: Shia...,0,0
3,3,3,30812,896087593,19200101,192001,1920,1920.0027,GOV,GOVERNOR,NaN,NaN,NaN,NaN,NaN,GOV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,141,Demonstrate or rally,141,14,3,-6.5,2,1,2,-2.087475,4,"Kannur, Kerala, India",IN,IN13,17698,11.8667,75.3653,-2092528,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,"Kerala, Kerala, India",IN,IN13,17699.0,10.0000,76.5000,-2100307,20200101063000,https://www.business-standard.com/article/pti-...,entry of 2 young women in sabarimala rahul gan...,\n\n\n\n\n\n\n\nEntry of 2 young women in Sab...,1,1
4,4,4,31666,896088447,19200101,192001,1920,1920.0027,GOV,MINIST,NaN,NaN,NaN,NaN,NaN,GOV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,112,"Accuse, not specified below",112,11,3,-2.0,6,1,6,-3.986711,1,India,IN,IN,NaN,20.0000,77.0000,IN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,India,IN,IN,NaN,20.0000,77.0000,IN,20200101064500,https://thehimalayantimes.com/nepal/minister-g...,minister gokul prasad baskota pledges to book ...,\n\n\n\n\n\n\n\n\nMinister Gokul Prasad Baskot...,0,0


In [52]:
data['Topic_nmf'].values.tolist().count(0)

4250

In [0]:
nmf_dataset=data[data['Topic_nmf']==0]

In [0]:
lda_nmf_dataset=nmf_dataset[(nmf_dataset['Topic_lda']==2) | (nmf_dataset['Topic_lda']==3)]

In [0]:
lda_nmf_dataset.to_csv('news-lda_nmf_data.csv')